In [ ]:
import requests
import json
from typing import List, Dict, Optional
import pandas as pd
import math
import time  # Importa a biblioteca time para adicionar atrasos

In [ ]:
# URL base da API
BASE_URL = "http://ec2-52-67-119-247.sa-east-1.compute.amazonaws.com:8000"
LOGIN_ENDPOINT = f"{BASE_URL}/login"


def login_and_get_token(username: str, password: str) -> Optional[str]:
    """
    Faz a requisição de login e retorna a token JWT.

    Args:
        username: O nome de usuário para login.
        password: A senha para login.

    Returns:
        A string da token JWT se o login for bem-sucedido, ou None em caso de falha.
    """
    dados_login = {
        "username": username,
        "password": password
    }

    try:
        print(f"Tentando logar em: {LOGIN_ENDPOINT}")
        response = requests.post(LOGIN_ENDPOINT, json=dados_login)

        if response.status_code == 200:
            print("Login bem-sucedido.")
            resposta_json = response.json()

            # ATENÇÃO: Confirme se a chave é 'access_token' ou outra na sua API.
            token_jwt = resposta_json.get("access_token")

            if token_jwt:
                return token_jwt
            else:
                print("ERRO: 'access_token' não encontrado na resposta.")
                print("Resposta da API:", resposta_json)
                return None
        else:
            print(f"Falha no login. Status HTTP: {response.status_code}")
            print("Detalhes:", response.text)
            return None

    except requests.exceptions.RequestException as e:
        print(f"ERRO DE REQUISIÇÃO: {e}")
        return None


if __name__ == '__main__':
  # AUTENTIQUE-SE COM SUAS CREDENCIAIS DE ACESSO
    USER = "USUARIO"
    PASS = "SENHA"

    token = login_and_get_token(USER, PASS)

    if token:
        print(f"\nToken obtida (para uso em outros arquivos): {token[:10]}...{token[-10:]}")
    else:
        print("\nFalha ao obter o token.")

Tentando logar em: http://ec2-52-67-119-247.sa-east-1.compute.amazonaws.com:8000/login
Login bem-sucedido.

Token obtida (para uso em outros arquivos): eyJhbGciOi...IwVC1qOU3g


In [ ]:
# --- Configurações da API ---
POKEMON_LIST_ENDPOINT = f"{BASE_URL}/pokemon"
POKEMON_DETAIL_ENDPOINT = f"{BASE_URL}/pokemon" # Será usado como POKEMON_DETAIL_ENDPOINT/{id}
PER_PAGE = 50       # Máximo de itens por página na lista

# --- Configurações de Controle de Requisições ---
DELAY_BETWEEN_LIST_PAGES = 0.05  # Atraso entre as requisições da lista paginada
DELAY_BETWEEN_DETAILS = 0.015     # Atraso entre as requisições individuais de detalhes
MAX_RETRIES = 5


def safe_request(url: str, headers: Dict, method: str = 'GET', max_retries: int = MAX_RETRIES) -> Optional[requests.Response]:
    """
    Função auxiliar para realizar requisições com tratamento de erro 429 (Rate Limit).
    """
    for attempt in range(max_retries):
        try:
            if method == 'GET':
                response = requests.get(url, headers=headers)
            else:
                # Adapte para outros métodos se necessário
                raise NotImplementedError("Método de requisição não implementado no safe_request")

            if response.status_code == 200:
                return response

            elif response.status_code == 429:
                # Tratamento de Rate Limit
                retry_after = response.headers.get('Retry-After')
                wait_time = int(retry_after) if retry_after and retry_after.isdigit() else (2 ** attempt)

                print(f"\nAVISO 429: Rate Limit. Tentativa {attempt + 1}. Aguardando {wait_time}s para {url}...")
                time.sleep(wait_time)

            elif response.status_code == 401:
                print(f"\nERRO 401: Não autorizado ao acessar {url}. Verifique o token.")
                return None

            else:
                # Outros erros HTTP (404, 500, etc.)
                print(f"\nERRO: Falha ao acessar {url}. Status HTTP: {response.status_code}. Resposta: {response.text}")
                return None # Falha sem possibilidade de retry (exceto 429)

        except requests.exceptions.RequestException as e:
            print(f"\nERRO DE CONEXÃO ao acessar {url}: {e}. Tentando novamente em {DELAY_BETWEEN_LIST_PAGES}s.")
            time.sleep(DELAY_BETWEEN_LIST_PAGES)

    print(f"\nFALHA CRÍTICA: Não foi possível acessar {url} após {max_retries} tentativas.")
    return None


def get_all_pokemon_base_info(token: str) -> List[Dict]:
    """
    Coleta a lista básica de {id, name} de todos os pokemons usando paginação.
    """
    base_info_list = []
    total_pokemons = None
    headers_autenticado = {"Authorization": f"Bearer {token}"}

    # 1. Obter o total de Pokémons
    response = safe_request(f"{POKEMON_LIST_ENDPOINT}?page=1&per_page={PER_PAGE}", headers_autenticado)
    if not response:
        return []

    initial_data = response.json()
    total_pokemons = initial_data.get("total")
    if total_pokemons is None:
        print("Erro: Chave 'total' não encontrada na resposta da API.")
        return []

    base_info_list.extend(initial_data.get("pokemons", []))

    total_pages = math.ceil(total_pokemons / PER_PAGE)
    print(f"Total de Pokémons: {total_pokemons}. Total de páginas: {total_pages}.")

    # 2. Coletar o restante das páginas
    for page_num in range(2, total_pages + 1):
        url_paginada = f"{POKEMON_LIST_ENDPOINT}?page={page_num}&per_page={PER_PAGE}"
        print(f"  -> Coletando lista da página {page_num}/{total_pages}...", end="\r")

        response = safe_request(url_paginada, headers_autenticado)

        if response and response.status_code == 200:
            data = response.json()
            base_info_list.extend(data.get("pokemons", []))
        elif response is None:
             # O safe_request já imprimiu a falha crítica. Abortamos a coleta da lista.
             return []

        time.sleep(DELAY_BETWEEN_LIST_PAGES) # Atraso entre páginas da lista

    print(f"\nColeta da lista básica concluída. {len(base_info_list)} IDs coletados.")
    return base_info_list


def get_all_pokemon_details(token: str) -> pd.DataFrame:
    """
    Coleta os IDs dos pokemons e faz uma requisição detalhada para cada um.
    """
    # 1. Obter a lista básica de IDs e nomes
    pokemon_base_info = get_all_pokemon_base_info(token)
    if not pokemon_base_info:
        return pd.DataFrame()

    all_details_list = []
    headers_autenticado = {"Authorization": f"Bearer {token}"}

    print("\nIniciando coleta de detalhes individuais...")

    # 2. Iterar sobre a lista e buscar os detalhes
    for index, info in enumerate(pokemon_base_info):
        pokemon_id = info.get("id")
        pokemon_name = info.get("name", "Nome Desconhecido")

        if pokemon_id is None:
            print(f"AVISO: Pokémon no índice {index} sem ID. Pulando.")
            continue

        url_detalhe = f"{POKEMON_DETAIL_ENDPOINT}/{pokemon_id}"

        # Monitora o progresso
        progress_msg = f"  -> Detalhes de {index + 1}/{len(pokemon_base_info)}: {pokemon_name} (ID: {pokemon_id})..."
        print(progress_msg, end='\r')

        response = safe_request(url_detalhe, headers_autenticado)

        if response and response.status_code == 200:
            # O JSON retornado já é o dicionário completo que queremos
            details = response.json()
            all_details_list.append(details)

        # 3. Pausa para evitar Rate Limit nas requisições individuais
        time.sleep(DELAY_BETWEEN_DETAILS)

    print(f"\nColeta de detalhes concluída. {len(all_details_list)} registros detalhados coletados.")

    # 4. Criar o DataFrame final
    if all_details_list:
        df_pokemons = pd.DataFrame(all_details_list)
        # O DataFrame conterá automaticamente todas as chaves do JSON (id, name, hp, attack, etc.)
        return df_pokemons
    else:
        return pd.DataFrame()


if __name__ == '__main__':
    # 1. Login e obtenção do token
    jwt_token = login_and_get_token(USER, PASS)

    if jwt_token:
        # 2. Coletar detalhes em massa e criar o DataFrame
        df_pokemon_detalhes = get_all_pokemon_details(jwt_token)

        if not df_pokemon_detalhes.empty:
            print(f"\nDataFrame final criado com sucesso! Total de linhas: {len(df_pokemon_detalhes)}")
            print("\nPrimeiras 5 linhas do DataFrame detalhado:")
            print(df_pokemon_detalhes.head())
            print("\nColunas e tipos de dados (info):")
            df_pokemon_detalhes.info()

            # Exemplo: Salvar para um arquivo CSV
            # df_pokemon_detalhes.to_csv('todos_pokemons_detalhados.csv', index=False)

        else:
            print("\nFalha na coleta de detalhes. O DataFrame final está vazio.")
    else:
        print("\nNão foi possível obter o token. A coleta de dados detalhados foi abortada.")

Tentando logar em: http://ec2-52-67-119-247.sa-east-1.compute.amazonaws.com:8000/login
Login bem-sucedido.
Total de Pokémons: 799. Total de páginas: 16.

Coleta da lista básica concluída. 799 IDs coletados.

Iniciando coleta de detalhes individuais...

Coleta de detalhes concluída. 799 registros detalhados coletados.

DataFrame final criado com sucesso! Total de linhas: 799

Primeiras 5 linhas do DataFrame detalhado:
   id           name    hp  attack  defense  sp_attack  sp_defense  speed  \
0   1      Bulbasaur  45.0    49.0     49.0         65          65   45.0   
1   2        Ivysaur  60.0    62.0     63.0         80          80   60.0   
2   3       Venusaur  80.0    82.0     83.0        100         100   80.0   
3   4  Mega Venusaur  80.0   100.0    123.0        122         120   80.0   
4   5     Charmander  39.0    52.0     43.0         60          50   65.0   

  generation legendary         types  
0          1     false  Grass/Poison  
1          1     false  Grass/Poison  

In [ ]:
df_pokemon_detalhes

,id,name,hp,attack,defense,sp_attack,sp_defense,speed,generation,legendary,types
0,1,Bulbasaur,45.0,49.0,49.0,65,65,45.0,1,false,Grass/Poison
1,2,Ivysaur,60.0,62.0,63.0,80,80,60.0,1,false,Grass/Poison
2,3,Venusaur,80.0,82.0,83.0,100,100,80.0,1,false,Grass/Poison
3,4,Mega Venusaur,80.0,100.0,123.0,122,120,80.0,1,false,Grass/Poison
4,5,Charmander,39.0,52.0,43.0,60,50,65.0,1,false,Fire
...,...,...,...,...,...,...,...,...,...,...,...
794,796,Diancie,50.0,100.0,150.0,100,150,50.0,6,true,Rock/Fairy
795,797,Mega Diancie,50.0,160.0,110.0,160,110,110.0,6,true,Rock/Fairy
796,798,Hoopa Confined,80.0,110.0,60.0,150,130,70.0,6,true,Psychic/Ghost
797,799,Hoopa Unbound,80.0,160.0,60.0,170,130,80.0,6,true,Psychic/Dark


In [ ]:
# Verificar se o dataframe possui valores nulos
print("Verificando valores nulos no DataFrame:")
display(df_pokemon_detalhes.isnull().sum())

Verificando valores nulos no DataFrame:


,0
id,0
name,0
hp,0
attack,0
defense,0
sp_attack,0
sp_defense,0
speed,0
generation,0
legendary,0


In [ ]:
df_pokemon_detalhes

In [ ]:
# Substitui a ocorrência de ',' para '/' na coluna de tipos
df_pokemon_detalhes['types'] = df_pokemon_detalhes['types'].str.replace(',', '/')

print("Primeiras 5 linhas da coluna 'types' após a substituição:")
display(df_pokemon_detalhes['types'].head())

Primeiras 5 linhas da coluna 'types' após a substituição:


,types
0,Grass/Poison
1,Grass/Poison
2,Grass/Poison
3,Grass/Poison
4,Fire


In [ ]:
# Algumas linhas possuem tipo duplicado, ex.: Bug/ Flying/Flying, esse trecho vai corrigir isso e a saída sera ex.: Bug/Flying
def remove_duplicate_types(types_string):
    """Removes duplicate types from a slash-separated string."""
    if isinstance(types_string, str):
        types_list = [t.strip() for t in types_string.split('/')]
        unique_types = []
        for t in types_list:
            if t not in unique_types:
                unique_types.append(t)
        return '/'.join(unique_types)
    return types_string

df_pokemon_detalhes['types'] = df_pokemon_detalhes['types'].apply(remove_duplicate_types)

print("Primeiras 10 linhas da coluna 'types' após a correção de duplicatas:")
display(df_pokemon_detalhes['types'].head(10))

print("\nVerificando o tipo para o Pokémon com ID 735:")
display(df_pokemon_detalhes[df_pokemon_detalhes['id'] == 735]['types'])

Primeiras 10 linhas da coluna 'types' após a correção de duplicatas:


,types
0,Grass/Poison
1,Grass/Poison
2,Grass/Poison
3,Grass/Poison
4,Fire
5,Fire
6,Fire/Flying
7,Fire/Dragon
8,Fire/Flying
9,Water



Verificando o tipo para o Pokémon com ID 735:


,types
733,Bug/Flying


In [ ]:
df_pokemon_detalhes.to_excel("pokemon.xlsx", index=False)

In [ ]:
# --- Configurações da API ---
COMBATS_ENDPOINT = f"{BASE_URL}/combats"
PER_PAGE = 100       # Valor de exemplo, ajuste se a API permitir mais ou menos

# --- Configurações de Controle de Requisições ---
DELAY_BETWEEN_PAGES = 0.015  # Atraso entre as requisições
MAX_RETRIES = 5


def safe_request(url: str, headers: Dict, max_retries: int = MAX_RETRIES) -> Optional[requests.Response]:
    """
    Função auxiliar para realizar requisições com tratamento de erro 429 (Rate Limit).
    """
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                return response

            elif response.status_code == 429:
                # Tratamento de Rate Limit
                retry_after = response.headers.get('Retry-After')
                wait_time = int(retry_after) if retry_after and retry_after.isdigit() else (2 ** attempt)

                print(f"\nAVISO 429: Rate Limit. Tentativa {attempt + 1}. Aguardando {wait_time}s para {url}...")
                time.sleep(wait_time)

            elif response.status_code == 401:
                print(f"\nERRO 401: Não autorizado ao acessar {url}. Verifique o token.")
                return None

            else:
                print(f"\nERRO: Falha ao acessar {url}. Status HTTP: {response.status_code}. Resposta: {response.text}")
                return None

        except requests.exceptions.RequestException as e:
            print(f"\nERRO DE CONEXÃO ao acessar {url}: {e}. Tentando novamente em {DELAY_BETWEEN_PAGES}s.")
            time.sleep(DELAY_BETWEEN_PAGES)

    print(f"\nFALHA CRÍTICA: Não foi possível acessar {url} após {max_retries} tentativas.")
    return None


def get_all_combats_to_dataframe(token: str) -> pd.DataFrame:
    """
    Coleta todos os combates do endpoint /combats, gerencia a paginação e cria um DataFrame
    com uma coluna de ID sequencial.
    """
    all_combats_data: List[Dict] = []
    total_combats = None
    headers_autenticado = {"Authorization": f"Bearer {token}"}

    # 1. Obter o total de combates e a primeira página
    url_initial = f"{COMBATS_ENDPOINT}?page=1&per_page={PER_PAGE}"
    response = safe_request(url_initial, headers_autenticado)

    if not response or response.status_code != 200:
        return pd.DataFrame()

    initial_data = response.json()
    total_combats = initial_data.get("total")

    if total_combats is None:
        print("Erro: Chave 'total' não encontrada na resposta da API.")
        return pd.DataFrame()

    current_combats = initial_data.get("combats", [])
    all_combats_data.extend(current_combats)

    total_pages = math.ceil(total_combats / PER_PAGE)
    print(f"Total de Combates: {total_combats}. Total de páginas estimadas: {total_pages}.")
    time.sleep(DELAY_BETWEEN_PAGES)

    # 2. Coletar o restante das páginas
    for page_num in range(2, total_pages + 1):
        url_paginada = f"{COMBATS_ENDPOINT}?page={page_num}&per_page={PER_PAGE}"
        print(f"  -> Coletando página {page_num}/{total_pages}. Combates coletados: {len(all_combats_data)}...", end="\r")

        response = safe_request(url_paginada, headers_autenticado)

        if response and response.status_code == 200:
            data = response.json()
            # ATENÇÃO: Acessa a lista de combates na chave 'combats'
            current_combats = data.get("combats", [])
            all_combats_data.extend(current_combats)
        elif response is None:
             # Falha crítica no safe_request, abortamos a coleta
             break

        time.sleep(DELAY_BETWEEN_PAGES) # Atraso entre páginas

    print(f"\nColeta de dados de combate concluída. Total de registros: {len(all_combats_data)}.")

    # 3. Criar o DataFrame e adicionar a coluna de ID sequencial
    if all_combats_data:
        df_combats = pd.DataFrame(all_combats_data)

        # Adiciona a coluna 'battle_id' (ID sequencial)
        # Usa range(1, len(df_combats) + 1) para começar em 1
        df_combats.insert(0, 'battle_id', range(1, len(df_combats) + 1))

        return df_combats
    else:
        return pd.DataFrame()


if __name__ == '__main__':
  # AUTENTIQUE-SE COM SUAS CREDENCIAIS DE ACESSO
    USER = "USUARIO"
    PASS = "SENHA"

    # 1. Login e obtenção do token
    jwt_token = login_and_get_token(USER, PASS)

    if jwt_token:
        # 2. Coletar dados de combate em massa e criar o DataFrame
        df_combats = get_all_combats_to_dataframe(jwt_token)

        if not df_combats.empty:
            print(f"\nDataFrame de Combates criado com sucesso! Total de linhas: {len(df_combats)}")
            print("\nPrimeiras 5 linhas do DataFrame:")
            print(df_combats.head())
            print("\nColunas e tipos de dados (info):")
            df_combats.info()

            # Exemplo: Salvar para um arquivo CSV
            # df_combats.to_csv('todos_combates.csv', index=False)

        else:
            print("\nFalha na coleta de combates. O DataFrame final está vazio.")
    else:
        print("\nNão foi possível obter o token. A coleta de dados de combate foi abortada.")

Tentando logar em: http://ec2-52-67-119-247.sa-east-1.compute.amazonaws.com:8000/login
Login bem-sucedido.
Total de Combates: 50000. Total de páginas estimadas: 500.

Coleta de dados de combate concluída. Total de registros: 50000.

DataFrame de Combates criado com sucesso! Total de linhas: 50000

Primeiras 5 linhas do DataFrame:
   battle_id first_pokemon second_pokemon winner
0          1           266            298    298
1          2           702            701    701
2          3           191            668    668
3          4           237            683    683
4          5           151            231    151

Colunas e tipos de dados (info):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   battle_id       50000 non-null  int64 
 1   first_pokemon   50000 non-null  object
 2   second_pokemon  50000 non-null  object
 3   winner 

In [ ]:
df_combats

,battle_id,first_pokemon,second_pokemon,winner
0,1,266,298,298
1,2,702,701,701
2,3,191,668,668
3,4,237,683,683
4,5,151,231,151
...,...,...,...,...
49995,49996,707,126,707
49996,49997,589,664,589
49997,49998,303,368,368
49998,49999,109,89,109


In [ ]:
# Verificar se o dataframe possui valores nulos
print("Verificando valores nulos no DataFrame:")
display(df_combats.isnull().sum())

Verificando valores nulos no DataFrame:


,0
battle_id,0
first_pokemon,0
second_pokemon,0
winner,0


In [ ]:
df_combats.to_excel("combat_pokemon.xlsx", index=False)